In [1]:
import os
from ftplib import FTP
import gzip
import io
import pandas as pd
import numpy as np

In [ ]:
pd.to_pickle(

In [12]:
processed_path = os.path.join('data','processed','stinson2019','weather_stn')
processed_path

'../data/processed/stinson2019/weather_stn'

In [2]:
weather_stn = pd.read_pickle('../data/processed/stinson2019/weather_stn')
closest_weather_stn = pd.read_pickle('../data/processed/stinson2019/closest_weather_stn')
tree_site = pd.read_pickle('../data/processed/stinson2019/norm_tables/tree_site')
tap_tree = pd.read_pickle('../data/processed/stinson2019/norm_tables/tap_tree')
tap_records = pd.read_pickle('../data/processed/stinson2019/norm_tables/tap_records')
dates = pd.read_pickle('../data/processed/stinson2019/norm_tables/dates')

In [3]:
# Create dataframe with all required info to connect weather station id to the number of years of sap measurements
df = (
    weather_stn.merge(closest_weather_stn.reset_index(), on="stn_id", how="right")
    .merge(tree_site.reset_index(), on="site", how="right")
    .merge(tap_tree.reset_index(), on="tree", how="right")
    .merge(tap_records.reset_index(), on="tap_id", how="right")
    .merge(dates.reset_index(), on="record_id", how="right")
)

# Create an populate table relating weather station ID with associated years of sap measurements
record_range = weather_stn.merge(closest_weather_stn.reset_index(), on="stn_id", how="right").set_index('site')
record_range['first_year'] = int()
record_range['last_year'] = int()

for site in closest_weather_stn.index:
    first_year = df[df.site==site].date.min().year
    last_year = df[df.site==site].date.max().year
    
    if first_year < record_range.loc[site,'start'].year:
        print(f"Earliest measurements at site {site} occured before weather station ID {record_range.loc[site,'stn_id']} began collecting data.")
        break
    else:
        record_range.loc[site,'first_year'] = first_year
        
    if first_year < record_range.loc[site,'start'].year:
        print(f"Latest measurements at site {site} occured after weather station ID {record_range.loc[site,'stn_id']} stopped collecting data.")
        break
    else:
        record_range.loc[site,'last_year'] = last_year

In [10]:
record_range.loc[:, ['stn_id', 'first_year', 'last_year']]

,stn_id,first_year,last_year
site,,,
INDU,726358-00384,2016,2017
SMM,724115-93757,2014,2017
DR,724117-63802,2016,2017
QC,716170-99999,2014,2017
HF,725085-54756,2012,2017
DOF,726116-94765,2014,2017


In [4]:
weather = pd.DataFrame(columns = ["stn_id", "datetime","air_temp"])

noaa_ftp = FTP("ftp.ncei.noaa.gov")
noaa_ftp.login()

for _, stn in record_range.iterrows():
    for year in range(stn.first_year,stn.last_year+1):

        # Generate filename based on selected station number and year and download
        # data from NOAA FTP site.
        filepath = "pub/data/noaa/" + str(year) + "/"
        filename = stn.stn_id + "-" + str(year) + ".gz"

        compressed_data = io.BytesIO()

        try:
            noaa_ftp.retrbinary("RETR " + filepath + filename, compressed_data.write)
        except error_perm as e_message:
            print(f"Error generated from NOAA FTP site for station {stn.stn_id} and year {year}: \n", e_message)
            noaa_ftp.quit()
            break

        # Unzip and process data line by line and extract variables of interest
        # The raw data file format is described here:
        # ftp://ftp.ncei.noaa.gov/pub/data/noaa/isd-format-document.pdf
        compressed_data.seek(0)
        stn_year_df = pd.DataFrame(
            columns=[
                "stn_id",
                "datetime",
                "air_temp",
            ]
        )
        with gzip.open(compressed_data, mode="rt") as stn_data:
            stn_data_df = pd.read_csv(stn_data, names=['data'])
            stn_year_df["datetime"] = pd.to_datetime(stn_data_df.data.str.slice(15,27))
            stn_year_df["air_temp"] = pd.to_numeric(stn_data_df.data.str.slice(87,92)) /10

        # Replace missing value indicators with NaNs
        stn_year_df = stn_year_df.replace(
            [999, 999.9, 9999.9], [np.nan, np.nan, np.nan]
        )

        stn_year_df.loc[:, "stn_id"] = stn.stn_id
        weather = weather.append(stn_year_df)
        
        
noaa_ftp.quit()

weather = weather.set_index("stn_id")

In [5]:
weather

,datetime,air_temp
stn_id,,
726358-00384,2016-01-01 00:15:00,-2.0
726358-00384,2016-01-01 00:35:00,-2.0
726358-00384,2016-01-01 00:55:00,-2.0
726358-00384,2016-01-01 01:15:00,-2.0
726358-00384,2016-01-01 01:35:00,-2.0
...,...,...
726116-94765,2017-12-31 19:53:00,-15.6
726116-94765,2017-12-31 20:53:00,-15.6
726116-94765,2017-12-31 21:53:00,-16.7
